### *I - Design a feedforward deep neural network (DNN) which consists of **three** hidden layers of 128 neurons each with ReLU activation function, and an output layer with sigmoid activation function. Apply dropout of probability **0.2** to each of the hidden layers.*

In [1]:
import tqdm
import time
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import torch
from torch import nn
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

from scipy.io import wavfile as wav

from sklearn import preprocessing
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, precision_score, recall_score, confusion_matrix

from common_utils import set_seed

# setting seed
set_seed()

1. Define the model class.

In [2]:
class MLP(nn.Module):

    def __init__(self, no_features, no_hidden, no_labels):
        super().__init__()
        self.mlp_stack = nn.Sequential(
            nn.Linear(no_features, no_hidden),
            nn.ReLU(),
            nn.Dropout(p=0.2),
            nn.Linear(no_hidden, no_hidden),
            nn.ReLU(),
            nn.Dropout(p=0.2),
            nn.Linear(no_hidden, no_hidden),
            nn.ReLU(),
            nn.Dropout(p=0.2),
            nn.Linear(no_hidden, no_labels),
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.mlp_stack(x)

## *II - Divide the dataset into a 80:20 ratio for training and testing. Use **appropriate** scaling of input features. We solely assume that there are only two datasets here: training & test.*

2. Split the dataset and do preprocessing.

In [3]:
from common_utils import split_dataset, preprocess_dataset

def preprocess(df):
    columns_to_drop = ['filename', 'label']
    test_size = 0.2
    random_state = 42

    X_train, y_train, X_test, y_test = split_dataset(df, columns_to_drop, test_size, random_state)

    X_train_scaled, X_test_scaled = preprocess_dataset(X_train, X_test)

    X_train_scaled = torch.tensor(X_train_scaled, dtype = torch.float32)
    y_train = torch.tensor(y_train, dtype = torch.float32)
    X_test_scaled = torch.tensor(X_test_scaled, dtype = torch.float32)
    y_test = torch.tensor(y_test, dtype = torch.float32)

    return X_train_scaled, y_train, X_test_scaled, y_test

df = pd.read_csv('/content/drive/MyDrive/a - csv file/simplified.csv')
df['label'] = df['filename'].str.split('_').str[-2]

X_train_scaled, y_train, X_test_scaled, y_test = preprocess(df)

## *III - Use the training dataset to train the model for 100 epochs. Use a mini-batch gradient descent with **‘Adam’** optimizer with learning rate of **0.001**, and **batch size = 128**. Implement early stopping with patience of **3**.*

3. Define a Pytorch Dataset and Dataloaders.  

In [4]:
class CustomDataset(Dataset):
    def __init__(self, X, y):
      self.X = X
      self.y = y

    def __len__(self):
      return len(self.X)

    def __getitem__(self, idx):
      return self.X[idx], self.y[idx]



def initialise_loaders(X_train_scaled, y_train, X_test_scaled, y_test):
    train_set = CustomDataset(X_train_scaled, y_train)
    test_set = CustomDataset(X_test_scaled, y_test)

    train_dataloader = DataLoader(train_set, batch_size = 128, shuffle = True)
    test_dataloader = DataLoader(test_set, batch_size = 128, shuffle = True)

    return train_dataloader, test_dataloader



train_dataloader, test_dataloader = initialise_loaders(X_train_scaled, y_train, X_test_scaled, y_test)

In [13]:
# Define Early Stopping class
class EarlyStopping:
    def __init__(self, patience, delta):
        """
        :param patience: How many epochs to wait for improvement
        :param delta: Minimum change to qualify as an improvement
        """
        self.patience = patience
        self.delta = delta
        self.best_loss = None
        self.counter = 0
        self.early_stop = False

    def __call__(self, val_loss):
        if self.best_loss is None:
            self.best_loss = val_loss
        elif val_loss < self.best_loss - self.delta:
            self.best_loss = val_loss
            self.counter = 0  # Reset the counter if there's an improvement
        else:
            self.counter += 1

        # Stop the training if patience is exceeded
        if self.counter >= self.patience:
            self.early_stop = True



# Initialize early stopping object
early_stopping = EarlyStopping(patience=5, delta=0.001)

4. Next, define the model, optimizer and loss function.

In [6]:
model = MLP(no_features = X_train_scaled.shape[1], no_hidden = 128, no_labels = 1)
optimizer = torch.optim.Adam(model.parameters(), lr = 0.001)
loss_fn = nn.BCELoss()

5. Train model for 100 epochs. Record down train and test accuracies. Implement early stopping.

In [14]:
num_epochs = 100

for epoch in range(num_epochs):

    model.train()
    correct_train_predictions = 0

    for batch_index, (X_train_scaled, y_train) in enumerate(train_dataloader):
        # Forward pass
        optimizer.zero_grad()
        y_train_predicted = model(X_train_scaled)
        loss = loss_fn(y_train_predicted, y_train.unsqueeze(1))

        # Backward pass and optimization
        loss.backward()
        optimizer.step()

        # Calculate train accuracies
        y_train_predicted_labels = (y_train_predicted >= 0.5).float()
        correct_train_predictions += (y_train_predicted_labels == y_train).sum().item()

    train_accuracy = correct_train_predictions / len(train_dataloader.dataset)

    print(f"Epoch {epoch+1}/{num_epochs}, Train Accuracy: {train_accuracy:.4f}")

    # Validation step
    model.eval()
    total_val_loss = 0.0
    correct_test_predictions = 0
    with torch.no_grad():
        for X_test_scaled, y_test in test_dataloader:
            # Calculate predicted labels
            y_test_predicted = model(X_test_scaled)

            # Calculate validation losses
            val_loss = loss_fn(y_test_predicted, y_test.unsqueeze(1))
            total_val_loss += val_loss.item()

            # Calculate test accuracies
            y_test_predicted_labels = (y_test_predicted >= 0.5).float()
            correct_test_predictions += (y_test_predicted_labels == y_test).sum().item()

        test_accuracy = correct_test_predictions / len(test_dataloader.dataset)

    print(f"Epoch {epoch+1}/{num_epochs}, Test Accuracy: {test_accuracy:.4f}")

    # Check for early stopping
    avg_val_loss = total_val_loss / len(test_dataloader)
    early_stopping(avg_val_loss)
    if early_stopping.early_stop:
        print("Early stopping triggered, stop training.")
        break

Epoch 1/100, Train Accuracy: 64.1689
Epoch 1/100, Test Accuracy: 63.7264
Epoch 2/100, Train Accuracy: 64.2425
Epoch 2/100, Test Accuracy: 63.7139
Epoch 3/100, Train Accuracy: 64.2406
Epoch 3/100, Test Accuracy: 64.2421
Epoch 4/100, Train Accuracy: 64.2722
Epoch 4/100, Test Accuracy: 63.4900
Epoch 5/100, Train Accuracy: 64.3897
Epoch 5/100, Test Accuracy: 63.8118
Epoch 6/100, Train Accuracy: 64.3805
Epoch 6/100, Test Accuracy: 63.7388
Epoch 7/100, Train Accuracy: 64.2734
Epoch 7/100, Test Accuracy: 63.7090
Epoch 8/100, Train Accuracy: 64.1839
Epoch 8/100, Test Accuracy: 63.7678
Epoch 9/100, Train Accuracy: 64.1858
Epoch 9/100, Test Accuracy: 63.6982
Epoch 10/100, Train Accuracy: 64.1815
Epoch 10/100, Test Accuracy: 63.6376
Epoch 11/100, Train Accuracy: 64.1968
Epoch 11/100, Test Accuracy: 64.0307
Epoch 12/100, Train Accuracy: 64.2260
Epoch 12/100, Test Accuracy: 63.5605
Early stopping triggered, stop training.


## *IV - Plot train and test accuracies and losses on training and test data against training epochs and comment on the line plots.*

6. Comment on line plots.

In [ ]:
# YOUR CODE HERE
answer = ""